In [ ]:
import re
from glob import glob

In [ ]:
def strip_format(txt):
    texts = re.findall(r'[A-Z]*',txt)
    return ' '.join(' '.join(texts).split())

def cleanup(txt):
    if len(txt) < 2:
        return ""
    if "LOADING" in txt:
        return ""
    if "listen to this genre right now!" in txt:
        return ""
    #if "--- RECORDSEPARATOR ---" in txt:
    #    return ""
    return txt

def cleanup_track(txt):
    if "artist:" in txt:
        return ""
    if "--- RECORDSEPARATOR ---" in txt:
        return ""
    if "track:" in txt:
        return ""
    return txt

#def extract_track(txt):
    

In [ ]:
# For Tracks:

files = sorted([i.replace('../v2.5_raw_data/','').replace('.swf','') for i in glob('../v2.5_raw_data/*.swf')])    

for file in files:
    
    playlist = []
    
    track = {}
    
    num_guess = 0
    
    for fname in glob('../v2.5_swf_data/'+file+'/*.txt'): # 902 multiline
        with open(fname) as f:
            txt = f.read()

            t = cleanup(txt)

            #if t is not "":
                #print (t)
                #print ("")
            
            #if str(txt).isdigit():
            #    num_guess = int(txt)
            
            if "artist:" in t:
                t = t.replace("\n--- RECORDSEPARATOR ---\n",'\t').split('\t')
                #print(t)
                track = {'artist':t[0].replace('artist:','').title(),
                         'title':t[1].replace('track:','').title(),
                         #'num': num_guess
                        }
                playlist.append(track)
                #print(track)
            
    
    for i, track in enumerate(playlist):
        print("{}\t{}\t{}\t{}".format(file, i+1, track['artist'], track['title']))

In [ ]:
# For Titles, and AKAs

files = sorted([i.replace('../v2.5_raw_data/','').replace('.swf','') for i in glob('../v2.5_raw_data/*.swf')])    

for file in files:

    parts = []
    
    for fname in glob('../v2.5_swf_data/'+file+'/*.txt'): # 902 multiline
        with open(fname) as f:
            txt = f.read()

            t = cleanup_track(cleanup(txt)).title()
            if t is not "":
                parts.append(t)
    
    if len(parts) < 2:
        parts.append("")
        
    if len(parts) < 2:
        parts.append("")
        
    print(file, "\t", parts[0], "\t", parts[1].replace('Aka',''))

In [ ]:
#[print(j) for j in sorted([i.replace('../v2.5_raw_data/','').replace('.swf','') for i in glob('../v2.5_raw_data/*.swf')])]

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv('../v2.5_genres.csv')
df.head()

In [ ]:
#for guide text descriptions:


for idx, r in df.iterrows():
    #print(r['type'], r['file'])
    
    with open('../v2.5_raw_data/'+r['genre']+'.txt') as f:
        desc = f.read()
        desc = desc[5:len(desc)-2] # get rid of text=, --
        #print(desc)
    
    if (r['node'] == r['title']):
        header = '##'
        if r['node'] in set(['House','Trance','Techno','Breakbeat','Jungle','Hardcore','Downtempo']):
            header = '#'
        print (header, r['node'])
    else:
        print ("##", r['node']+":", r['title'])

    
    type_of = ""    
    if str(r['scene']) != 'nan':
        type_of = "in the " + r['scene'] + " group,"
    
    if str(r['aka']) != 'nan':
        print("*aka: ", r['aka'].strip()+"*", type_of, 'emerged in', r['decade'])
    else:
        print(type_of)
    print()
    print(desc)
    print()

In [ ]:
# Add descriptions to CSV
df = pd.read_csv('../v2.5_genres.csv')

for idx, r in df.iterrows():
    #print(r['type'], r['file'])
    
    with open('../v2.5_raw_data/'+r['genre']+'.txt') as f:
        desc = f.read()
        desc = desc[5:len(desc)-2] # get rid of text=, --
        desc = desc.strip()

        df.at[idx,'description'] = desc
        
        #print(desc)
        
        #print('\n' in desc)
        #if '\n' in desc:
        #    print(desc)

#df.to_csv('../v2.5_genres.csv', index=False)
df.sample(10)

In [ ]:
# https://stackoverflow.com/a/33869154/11090908
def pandas_df_to_markdown_table(df):
    from IPython.display import Markdown, display
    fmt = ['---' for i in range(len(df.columns))]
    df_fmt = pd.DataFrame([fmt], columns=df.columns)
    df_formatted = pd.concat([df_fmt, df])
    #display(Markdown(df_formatted.to_csv(sep="|", index=False)))
    return df_formatted.to_csv(sep="|", index=True)

In [ ]:
#df = pd.read_csv('../v2.5_genres.csv',dtype=str)
#df = pd.read_csv('../v2.5_links.csv',dtype=str)
#df = pd.read_csv('../v2.5_tracks.csv',dtype=str)

#df = pd.read_csv('../v3_genres.csv',dtype=str)
#df = pd.read_csv('../v3_links.csv',dtype=str)
df = pd.read_csv('../v3_tracks.csv',dtype=str)

df2 = df.describe(include='all').fillna(" ")
print(pandas_df_to_markdown_table(df2))
df2